In [31]:
DL = '/Users/damiansp/Downloads'

In [32]:
!rm -rf $DL/*

In [95]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [96]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMGN', 'AMZN',
    'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD', 'DDOG',
    'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED',
    'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD', 'LRCX', 'LSPD', 'LULU', 'MDB',
    'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA', 'ODFL', 'OKTA',
    'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON', 'PYPL', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP',
    'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TSLA', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN',
    'ZM']
LINGERERS = [
    'AMEH', 'AMRK', 'APT', 'BTTR', 'BYRN', 'CIZN', 'CUBE', 'DTST',
    'DYAI', 'EP', 'FCPT', 'FCUV', 'GBOX', 'GME', 'HMY', 'IEI', 'INTZ',
    'KREF', 'MCVT', 'MICS', 'MNMD', 'OMQS', 'PETV', 'POWW', 'PRPH',
    'RVP', 'VIRT', 'WAVD', 'WKHS', 'ZYXI']
BEST_WEIGHTED = [
    'AMRC', 'COOP', 'CSIQ', 'CTSH', 'CVE', 'ENPH', 'EPSN', 'FKWL',
    'GCBC', 'GGB', 'GGE', 'LFMD', 'NTAP', 'PACB', 'SNEX', 'SOBR', 'TJX',
    'UTHR', 'VTSI']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [97]:
!ls $DL

Portfolio_Positions_Oct-14-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_10_14.xls
clean.csv


In [98]:
#!head -5 $DL/Positions.csv

In [99]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [100]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [101]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [102]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [103]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,1383.8
ABMD,1495.68
ABNB,218.32
ADBE,3743.22
ADYEY,--


In [104]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,1384
ABMD,1496
ABNB,218
ADBE,3743
ADYEY,--


In [105]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [110]:
#et_df.loc[et_df.ET == '--']
et_df.loc['ADYEY', 'ET'] = 12.25

In [111]:
check_symbols(et_df)

All clear


In [112]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,1384
ABMD,1496
ABNB,218
ADBE,3743
ADYEY,12


### Fidelity

In [113]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Oct-14-2022.csv'

In [114]:
!ls $DL

Portfolio_Positions_Oct-14-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_10_14.xls
clean.csv


In [115]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-16-", "-17-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,2247.3,$1.00,$0.00,$2247.30,NaN,NaN,NaN,NaN,6.97%,NaN,NaN,Cash
1,218520300,Rollover IRA,ABMD,ABIOMED INC COM USD0.01,4.0,$249.28,-$8.80,$997.12,-$35.20,-3.41%,-$21.57,-2.12%,3.09%,$1018.69,$254.67,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.0,$12.25,-$0.076,$159.25,-$0.99,-0.62%,-$55.44,-25.83%,0.49%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.0,$55.94,-$3.00,$1006.92,-$54.00,-5.09%,-$339.99,-25.25%,3.12%,$1346.91,$74.83,Cash
4,218520300,Rollover IRA,AMZN,AMAZON.COM INC,2.0,$106.90,-$5.63,$213.80,-$11.26,-5.01%,-$62.97,-22.76%,0.66%,$276.77,$138.39,Cash


In [116]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,Rollover IRA,ABMD,997
2,Rollover IRA,ADYEY,159
3,Rollover IRA,AMD,1007
4,Rollover IRA,AMZN,214
5,Rollover IRA,AXON,358


In [117]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ABMD,997
ADYEY,159
AMD,1007
AMZN,214
AXON,358


In [118]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [119]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,1384,0,5535,0
ABMD,1496,997,3241,0
ABNB,218,0,546,0
ADBE,3743,0,0,0
ADYEY,12,159,2487,86


### TD Ameritrade

In [120]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [121]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [122]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [123]:
td = parse_file(path)
td.head()

,TD
AMEH,63.00
AMRK,105.76
APT,12.09
BTTR,119.88
BYND,13.32


In [124]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,1384,0,5535,0,0
ABMD,1496,997,3241,0,0
ABNB,218,0,546,0,0
ADBE,3743,0,0,0,0
ADYEY,12,159,2487,86,0


In [125]:
out.to_csv(f'{DL}/clean.csv')

In [126]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*